In [1]:
# conda install -n sim_search ipykernel --update-deps --force-reinstall
import pandas as pd
import numpy as np

from utils.search import return_res
from utils.data_process import input_process, res_process
from utils.search import do_search
# from utils.plot import plotly_plot
import matplotlib.pyplot as plt
import talib
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [16]:
from re import template
import pandas as pd
from utils.algo.calculation import get_fbna
from dash_bootstrap_templates import load_figure_template
import talib
import plotly.express as px

# Test

In [17]:
templates = ["solar"]
load_figure_template(templates)

In [18]:
def plot(df, s, e):
    df = pd.DataFrame(df)
    df['k'], df['d'] = talib.STOCH(df['High'], df['Low'], df['Close'])
    df['k'].fillna(value=0, inplace=True)
    df['d'].fillna(value=0, inplace=True)

    df['diff'] =df['Close'] - df['Open']
    df.loc[df['diff']>=0, 'color'] = 'green'
    df.loc[df['diff']<0, 'color'] = 'red'
    l1,l2,l3,l4,l5 = get_fbna(min(df['Low'][s:e]),max(df['High'][s:e]))


    fig = make_subplots(

    rows = 8, cols = 1,

    specs = [[{"rowspan": 4, "secondary_y": True}],
            [None],
            [None],
            [None],
            [{"rowspan":2}],
            [None],
            [{"rowspan":2}],
            [None]],

    print_grid=False, shared_xaxes=True, vertical_spacing=0.05
)
##########################################################

    # condle
    fig.add_trace(go.Candlestick(x=df.index[s:e],
                                open=df['Open'][s:e],
                                high=df['High'][s:e],
                                low=df['Low'][s:e],
                                close=df['Close'][s:e],
                                name="Price", 
                                ), secondary_y=False, row = 1, col = 1)
    # candle partition
    fig.update_yaxes(range=[min(df['Low'][s:e])*0.975, max(df["High"][s:e]*1.025)], row=1, col=1, title_text = "Candle")
    fig.add_hline(y = l1,line_dash="dot",row = 1, col=1, line_color = '#ff7f0e')
    fig.add_hline(y = l2,line_dash="dot",row = 1, col=1, line_color = '#8c564b')
    fig.add_hline(y = l3, line_dash="dot",row = 1, col=1, line_color ='#9467bd')
    fig.add_hline(y = l4, line_dash="dot", row = 1, col=1, line_color = '#bcbd22')
    fig.add_hline(y = l5, line_dash="dot", row = 1, col=1, line_color = '#1f77b4')


    # volume subplot
    fig.add_trace(go.Bar(x=df.index[s:e], y=df['Volume'][s:e], name='Volume', marker={'color':df['color']}),  row = 5, col = 1)
    fig.update_yaxes(title_text = "Vloume", row=5, col=1)

    # KD subplot
    fig.add_trace(go.Scatter(x=df.index[s:e], y = df["k"][s:e], name = "k", mode='lines'), row=7, col=1)
    fig.add_trace(go.Scatter(x=df.index[s:e], y = df["d"][s:e], name = "d", mode='lines'), row=7, col=1)
    fig.update_yaxes(title_text = "KD", row=7, col=1)


    fig.update_layout(autosize=False,
        width=1000*0.7,
        height=700*0.7,title_font_size = 1,xaxis_rangeslider_visible=False, xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=False) 

    fig.update_layout(template="solar")

    return fig

In [4]:
df = pd.read_csv('./data/2330TW.csv')

In [5]:
s = 2497 # import your time here
e = 2518

In [21]:
plot(df, s = s, e= e)

# Display Whole Wave

In [8]:
df['diff'] =df['Close'] - df['Open']
df.loc[df['diff']>=0, 'color'] = 'green'
df.loc[df['diff']<0, 'color'] = 'red'

In [25]:
fig = go.FigureWidget(

    go.Candlestick(
        x = df['Date'],
        open = df['Open'],
        close = df['Close'],
        high = df['High'],
        low = df['Low'],
    ),
    
)
# fig.update_xaxes(range=('2010-01-02','2011-12-25'))
def zoom(xrange):
    xrange_zoom_min, xrange_zoom_max = fig.layout.xaxis.range[0], fig.layout.xaxis.range[1]
    yaxis_range = df.loc[df['Date'].between(xrange_zoom_min,xrange_zoom_max), 'Close']
    padding = (yaxis_range.max() - yaxis_range.min()) / 16
    
    fig.layout.yaxis.range = [yaxis_range.min() - padding, yaxis_range.max() + padding]
    fig.layout.title = f"Setting new range to {fig.layout.yaxis.range}"    


fig.layout.on_change(zoom, fig)


TypeError: object of type 'FigureWidget' has no len()

In [ ]:
!pip install ipywidgets